In [1]:
from openai import OpenAI
import json
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
import urllib.parse
import random
import csv
import time

In [3]:
# GPT_MODEL = "gpt-3.5-turbo-0125"
GPT_MODEL = "gpt-4o-2024-05-13"

URL_GEOCODING = "https://geocoding-by-api-ninjas.p.rapidapi.com/v1/geocoding"
URL_WEATHER = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"
URL_WORKOUT = "https://work-out-api1.p.rapidapi.com/search"
URL_ACTOR_ID  = "https://moviesminidatabase.p.rapidapi.com/actor/imdb_id_byName/"
URL_MOVIE = "https://moviesminidatabase.p.rapidapi.com/movie/byActor/"
URL_MACROS = "https://fitness-calculator.p.rapidapi.com/macrocalculator"
URL_FIND_BY_NUTRIENTS = "https://api.spoonacular.com/recipes/findByNutrients"
URL_CLASSIFY = "https://api.spoonacular.com/food/images/classify"
URL_COMPLEX_SEARCH = "https://api.spoonacular.com/recipes/complexSearch"
URL_EXTRACT_RECIPE_FROM_URL = "https://api.spoonacular.com/recipes/extract"
URL_SEARCH_INGREDIENTS = "https://api.spoonacular.com/food/ingredients/search"
URL_CONVERT_AMOUNT = "https://api.spoonacular.com/recipes/convert"
URL_COMPLEX_SEARCH_RAPID = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/complexSearch"

LONG_LANG_KEY = ""
RAPID_KEY = ""
SPOONACULAR_KEY = ""

OPENAI_API_KEY = ""
client = OpenAI(api_key=OPENAI_API_KEY)

# Spoonacular Credentials
USER_HASH = ""
USERNAME = ""

SPOONACULAR_TOOLS = ["find_by_nutrients", "recipe_search", "extract_recipe", "search_ingredients", "get_ingredient_nutrition", "convert_amounts"]

Description of Tools

In [4]:
# List of all 15 tools handed over to the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_longitude_and_latitude",
            "description": "Returns the longitude and latitude of the given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city",
                    },
                    "country": {
                        "type": "string",
                        "description": "The name of the country",
                    },
                },
                "required": ["city", "country"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description": "Returns the current temperature for a given location specified by its longitude and latitude",
            "parameters": {
                "type": "object",
                "properties": {
                    "longitude": {
                        "type": "number",
                        "description": "The longitude of the location"
                    },
                    "latitude": {
                        "type": "number",
                        "description": "The latitude of the location"
                    }
                },
                "required": ["longitude", "latitude"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_workout",
            "description": "Returns a list of exercises based on the specified muscle groups, equipment, and intensity level",
            "parameters": {
                "type": "object",
                "properties": {
                    "muscles": {
                        "type": "string",
                        "description": "The specific muscle targeted by the exercises (e.g., 'Biceps', 'Legs')"
                    },
                    "equipment": {
                        "type": "string",
                        "description": "The equipment for the exercises (e.g., 'Dumbbells', 'Barbell')"
                    },
                    "intensity_level": {
                        "type": "string",
                        "description": "The intensity level of the exercises (e.g., 'Beginner', 'Intermediate', 'Expert')"
                    }
                },
                "required": ["muscles", "intensity_level"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_actor_id",
            "description": "Returns the ID of the specified actor",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the actor"
                    }
                },
                "required": ["name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_by_actor_id",
            "description": "Returns a random movie title featuring the specified actor by their IMDb ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "actor_id": {
                        "type": "string",
                        "description": "The IMDb ID of the actor"
                    }
                },
                "required": ["actor_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_macros",
            "description": "Returns the macronutrient needs (calories, protein, fat, carbs) based on age, gender, height, weight, activity level, and fitness goal",
            "parameters": {
                "type": "object",
                "properties": {
                    "age": {
                        "type": "number",
                        "description": "The age of the individual"
                    },
                    "gender": {
                        "type": "string",
                        "description": "The gender of the individual (e.g., 'male', 'female')"
                    },
                    "height": {
                        "type": "number",
                        "description": "The height of the individual in centimeters"
                    },
                    "weight": {
                        "type": "number",
                        "description": "The weight of the individual in kilograms"
                    },
                    "activity_level": {
                        "type": "number",
                        "description": "The activity level of the individual (e.g., 1 for sedentary, 2 for lightly active, 3 for moderately active, 4 for very active, 5 for extra active)"
                    },
                    "goal": {
                        "type": "string",
                        "description": "The fitness goal of the individual (e.g., 'lose', 'maintain', 'gain')"
                    }
                },
                "required": ["age", "gender", "height", "weight", "activity_level", "goal"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_calculation",
            "description": "Performs a basic arithmetic operation (addition, subtraction, multiplication, or division) on two numbers based on the specified calculation type",
            "parameters": {
                "type": "object",
                "properties": {
                    "a_number": {
                        "type": "number",
                        "description": "The first number for the calculation"
                    },
                    "b_number": {
                        "type": "number",
                        "description": "The second number for the calculation"
                    },
                    "calc_type": {
                        "type": "string",
                        "description": "The type of calculation to perform ('add', 'subtract', 'multiply', 'divide')"
                    }
                },
                "required": ["a_number", "b_number", "calc_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "find_by_nutrients",
            "description": "Returns a specified number of recipes that matches the specified nutrient criteria",
            "parameters": {
                "type": "object",
                "properties": {
                    "calories": {
                        "type": "number",
                        "description": "The minimum number of calories"
                    },
                    "protein": {
                        "type": "number",
                        "description": "The minimum amount of protein in grams"
                    },
                    "carbs": {
                        "type": "number",
                        "description": "The minimum amount of carbohydrates in grams"
                    },
                    "fat": {
                        "type": "number",
                        "description": "The minimum amount of fat in grams"
                    },
                    "number": {
                        "type": "number",
                        "description": "The number of different recipes that are returned"
                    }
                },
                "required": ["calories", "protein", "carbs", "fat", "number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "recipe_search",
            "description": "Searches for a number of recipes based on a combination of arguments. It requires at least one argument.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query for finding a recipe. The default value is None"
                    },
                    "add_recipe_nutrition": {
                        "type": "boolean",
                        "description": "If True, adds nutritional information to the recipe. If False, it doesn't add nutritional information. The default value is False"
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "Which type of cuisine the recipe is in. The default value is None"
                    },
                    "recipe_type": {
                        "type": "string",
                        "description": "Which kind of recipe. Possible recipe_types include breakfast, appetizer, soup and so on. The default value is None"
                    },
                    "number": {
                        "type": "number",
                        "description": "The number of different recipes that are returned. The default value is 1"
                    },
                    "include_ingredients": {
                        "type": "string",
                        "description": "A comma-separated list of ingredients that should be included in the recipe. The default value is None"
                    },
                    "exclude_ingredients": {
                        "type": "string",
                        "description": "A comma-separated list of ingredients that should be excluded in the recipe. The default value is None"
                    },
                    "show_ingredients": {
                        "type": "boolean",
                        "description": "If True, shows the ingredients in the recipe. The default value is False"
                    },
                    "offset": {
                        "type": "number",
                        "description": "Number of results to skip. The default value is None"
                    }
                },
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "extract_recipe",
            "description": "Extracts information from a recipe url about the recipe, including information about ingredients, their amounts and the time needed to make the recipe.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the recipe to extract ingredients from"
                    }
                },
                "required": ["url"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_ingredients",
            "description": "Searches for an ingredient based on a query and returns the ingredient's ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query for finding an ingredient"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_ingredient_nutrition",
            "description": "Returns the nutritional information for a specified amount of an ingredient based on its ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "ingredient_id": {
                        "type": "number",
                        "description": "The ID of the ingredient"
                    },
                    "amount": {
                        "type": "number",
                        "description": "The amount of the ingredient"
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit of measurement for the ingredient amount (e.g., 'g', 'oz')"
                    }
                },
                "required": ["ingredient_id", "amount", "unit"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "convert_amounts",
            "description": "Converts the amount of an ingredient from one unit to another",
            "parameters": {
                "type": "object",
                "properties": {
                    "ingredient_name": {
                        "type": "string",
                        "description": "The name of the ingredient"
                    },
                    "source_amount": {
                        "type": "number",
                        "description": "The amount of the ingredient to convert"
                    },
                    "source_unit": {
                        "type": "string",
                        "description": "The unit of the ingredient amount to convert from (e.g., 'cups', 'grams')"
                    },
                    "target_unit": {
                        "type": "string",
                        "description": "The unit of the ingredient amount to convert to (e.g., 'cups', 'grams')"
                    }
                },
                "required": ["ingredient_name", "source_amount", "source_unit", "target_unit"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "post_to_shopping_list",
            "description": "Adds an item to a user's shopping list",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {
                        "type": "string",
                        "description": "The item to add to the shopping list"
                    }
                },
                "required": ["item"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_items_from_storage",
            "description": "Returns a list of items from the storage.csv file, including their names, amounts, units, and prices.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {
                        "type": "string",
                        "description": "the storage file to retrieve the items from (storage.csv)"
                    }
                },
                "required": []
            }
        }
    }
]

Tools implemented as Functions

In [5]:
def get_longitude_and_latitude(city, country):
    """
    :param city: City name 
    :param country: Country name
    :return: longitude and latitude value
    """
    querystring = {"city": city, "country": country}
    headers = {
        "X-RapidAPI-Key": LONG_LANG_KEY,
        "X-RapidAPI-Host": "geocoding-by-api-ninjas.p.rapidapi.com"
    }

    try:
        response = requests.get(URL_GEOCODING, headers=headers, params=querystring)
        data = response.json()
        
        if data:  # Checking if there is data returned
            lat_lon_data = [{"latitude": item["latitude"], "longitude": item["longitude"]} for item in data]
            return json.dumps(lat_lon_data)
        else:
            return json.dumps([])  # Return an empty list if no data is found
    except Exception as e:
        error_message = str(e)
        print("Error in get_longitude_and_latitude with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def get_temperature(longitude, latitude):
    """
    :param longitude: longitude value 
    :param latitude: latitude value
    :return: temperature
    """
    querystring = {"lon":longitude, "lat":latitude}
    headers = {
        "X-RapidAPI-Key": RAPID_KEY,
        "X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
    }
    
    try:
        response = requests.get(URL_WEATHER, headers=headers, params=querystring)
        data = response.json()
        
        return json.dumps({"temperature": data["temp"]})
    except Exception as e:
        error_message = str(e)
        print("Error in get_temperature with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def get_workout(muscles, intensity_level, equipment = None):
    """
    :param muscles: specific muscle to target
    :param intensity_level: intensity of exercise
    :param equipment: type of equipment needed for the exercise
    :return: name, volume, explanation of exercise
    """
    querystring = {"Muscles":muscles,"Equipment":equipment,"Intensity_Level":intensity_level}
    headers = {
        "X-RapidAPI-Key": RAPID_KEY,
        "X-RapidAPI-Host": "work-out-api1.p.rapidapi.com"
    }
    
    try:
        response = requests.get(URL_WORKOUT, headers=headers, params=querystring)
        data = response.json()
        if data:
            workout_data = [{"Name": item["WorkOut"], "Volume": item["Beginner Sets"], "Explanation": item["Long Explanation"]} for item in data]
            return json.dumps(workout_data)
        else:
            return json.dumps([])
    except Exception as e:
        error_message = str(e)
        print("Error in get_workout with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def get_actor_id(name):
    """
    :param name: name of the actor 
    :return: actor id of the actor
    """
    encoded_name = urllib.parse.quote(name)
    full_url = f"{URL_ACTOR_ID}{encoded_name}"
    headers = {
        "X-RapidAPI-Key": RAPID_KEY,
        "X-RapidAPI-Host": "moviesminidatabase.p.rapidapi.com"
    }
    
    try:
        response = requests.get(full_url, headers=headers)
        data = response.json()
        return json.dumps({"actor_id": data['results'][0]['imdb_id']})
    except Exception as e:
        error_message = str(e)
        print("Error in get_actor_id with following parameters: " + json.dumps(encoded_name) + " " + error_message)
        
def get_movie_by_actor_id(actor_id):
    """
    :param actor_id: actor id of an actor 
    :return: a random movie with the specified actor
    """
    full_url = f"{URL_MOVIE}{actor_id}"
    
    headers = {
	"X-RapidAPI-Key": RAPID_KEY,
	"X-RapidAPI-Host": "moviesminidatabase.p.rapidapi.com"
    }
    
    try:
        response = requests.get(full_url, headers=headers)
        data = response.json()
        
        results = data["results"]
        movie_entries = [item[0] for item in results if 'title' in item[0]]
        random_movie = random.choice(movie_entries)
        return json.dumps(random_movie["title"])
    except Exception as e:
        error_message = str(e)
        print("Error in get_movie_by_actor_id with following parameters: " + json.dumps(full_url) + " " + error_message)
        
def get_macros_original(age, gender, height, weight, activity_level, goal):
   querystring = {"age":age,"gender":gender,"height":height,"weight":weight,"activitylevel":activity_level,"goal":goal}

   headers = {
       "X-RapidAPI-Key": RAPID_KEY,
       "X-RapidAPI-Host": "fitness-calculator.p.rapidapi.com"
   }

   try:
       response = requests.get(URL_MACROS, headers=headers, params=querystring)
       data = response.json()

       results = {
           "calorie": data['data']['calorie'],
           "protein": data['data']['balanced']['protein'],
           "fat": data['data']['balanced']['fat'],
           "carbs": data['data']['balanced']['carbs'],
       }
       return json.dumps(results)
   except Exception as e:
       error_message = str(e)
       print("Error in get_macros with following parameters: " + json.dumps(querystring) + " " + error_message)
       
# hard coded as API stopped working
def get_macros(age, gender, height, weight, activity_level, goal):
    """
    :param age: age of person
    :param gender: gender of person
    :param height: height in cm of person
    :param weight: weight in kg of person
    :param activity_level: activity level of person
    :param goal: fitness goal of person
    :return: macronutrient need
    """
        
    if activity_level > 2:
        results = {
            "calorie": 2564,
            "protein": 159,
            "fat": 75,
            "carbs": 314,
        }
    elif activity_level > 1:
        results = {
            "calorie": 2364,
            "protein": 139,
            "fat": 65,
            "carbs": 304,
        }
    else:
        results = {
            "calorie": 2000,
            "protein": 80,
            "fat": 40,
            "carbs": 200,
        }            
    return json.dumps(results)

def get_calculation(a_number, b_number, calc_type):
    """
    :param a_number: first number 
    :param b_number: second number
    :param calc_type: type of calculation
    :return: result of calculation
    """
    # Check the type of calculation and perform it accordingly
    if calc_type == "add":
        return a_number + b_number
    elif calc_type == "subtract":
        return a_number - b_number
    elif calc_type == "multiply":
        return a_number * b_number
    elif calc_type == "divide":
        # Handle division carefully to avoid dividing by zero
        if b_number == 0:
            return "Error: Division by zero is not allowed"
        else:
            return a_number / b_number
    else:
        return "Invalid calculation type specified"
    
def find_by_nutrients(calories, protein, carbs, fat, number):
    """
    :param calories: minimum calories wanted
    :param protein: minimum protein wanted
    :param carbs: minimum carbs wanted
    :param fat: minimum fat wanted
    :param number: number of recipies
    :return: id, title, image, calories, protein, fat, and carbs of recipe
    """
    querystring = {"minCalories":calories, "minProtein":protein, "minCarbs":carbs, "minFat":fat, "number":number}
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/findByNutrients"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url, headers=headers, params=querystring)
        data = response.json()
        return json.dumps(data)
    except Exception as e:
        error_message = str(e)
        print("Error in find_by_nutrients with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def recipe_search(query=None, add_recipe_nutrition=False, cuisine=None, recipe_type=None, number=1, include_ingredients=None, exclude_ingredients=None, show_ingredients=False, offset=None):
    """
    :param query: search query to find a recipe
    :param add_recipe_nutrition: adds nutritional information
    :param cuisine: type of cuisine
    :param recipe_type: type of recipe like breakfast or dinner
    :param number: number of recipes to return
    :param include_ingredients: list of ingredients the recipe needs to include
    :param exclude_ingredients: list of ingredients the recipe needs to exclude
    :param show_ingredients: adds ingredients to the result
    :param offset: number of results to skip
    :return: information about recipe like title and what is specified
    """
    
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/complexSearch"
    
    # Create a dictionary of all parameters
    querystring = {
        "query": query,
        "addRecipeNutrition": add_recipe_nutrition,
        "cuisine": cuisine,
        "type": recipe_type,
        "number": number,
        "fillIngredients": show_ingredients,
        "excludeIngredients": exclude_ingredients,
        "includeIngredients": include_ingredients,
        "offset": offset,
        "ignorePantry": False
    }
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    
    # Remove any parameters that are None
    querystring = {k: v for k, v in querystring.items() if v is not None}
    
    try:
        response = requests.get(url=url, headers=headers, params=querystring)
        data = response.json()
        results = []
        
        try:
            for result in data["results"]:
                if result["title"]:
                    results.append(result["title"])
                    
                if add_recipe_nutrition:
                    if result["servings"] is not None:
                        servings = "servings: " + str(result["servings"])
                        results.append(servings)
                    # Add nutrients
                    nutrients = result["nutrition"]["nutrients"]
                    interested_nutrients = {"Fat", "Calories", "Protein", "Carbohydrates"}
                        
                    for nutrient in nutrients:
                        if nutrient['name'] in interested_nutrients:
                           results.append(nutrient)
                        
                if show_ingredients:
                    for ingredient in result["missedIngredients"]:
                        ingredients = {"id": ingredient["id"], "name": ingredient["name"],
                                       "amount": ingredient["amount"], "unit": ingredient["unit"]}
                        results.append(ingredients)
                    
            return json.dumps(results)
        except Exception as e:
            error_message = str(e)
            print("Error while collecting specific information for the following parameters: " + json.dumps(querystring) + " " + error_message)
            return json.dumps(data)
    except Exception as e:
        error_message = str(e)
        print("Error in recipe_search with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def extract_recipe(url):
    """
    :param url: recipe url 
    :return: title, minutes to prepare and cook
    """
    querystring = {"url":url}
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/extract"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url=url, headers=headers, params=querystring)
        data = response.json()
        #print(data)
        print()
        ingredients = data['extendedIngredients']
        result = []
        for ingredient in ingredients:
            temp = {"name":ingredient['original'],"amount":ingredient['amount'],"unit":ingredient['unit']}
            result.append(temp)
        preparation_minutes = data['preparationMinutes']
        cooking_minutes = data['cookingMinutes']
        title = data['title']
        result.append({"preparationMinutes":preparation_minutes, "cookingMinutes":cooking_minutes, "title":title})
    
        return json.dumps(result)
    except Exception as e:
        error_message = str(e)
        print("Error in extract_recipe with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def search_ingredients(query):
    """
    :param query: name of ingredient
    :return: ingredient id
    """
    querystring = {"query":query}
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/ingredients/search"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }

    try:
        response = requests.get(url=url, headers=headers, params=querystring)
        return response.json()["results"][0]["id"]
    except Exception as e:
        error_message = str(e)
        print("Error in search_ingredients with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def get_ingredient_nutrition(ingredient_id, amount, unit):
    """
    :param ingredient_id: ingredient id 
    :param amount: ingredient amount
    :param unit: ingredient unit
    :return: calories, fat, protein, carbs of ingredient
    """
    querystring = {"amount":amount, "unit":unit}  
    url = f"https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/food/ingredients/{str(ingredient_id)}/information"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url=url, headers=headers, params=querystring)
        data = response.json()
        nutrients = data['nutrition']['nutrients']
        result = []
        interested_nutrients = {"Fat", "Calories", "Protein", "Carbohydrates"}
       
        for nutrient in nutrients:
            if nutrient['name'] in interested_nutrients:
                nutrient.pop('percentOfDailyNeeds', None)
                result.append(nutrient)
        
        return json.dumps(result)
    except Exception as e:
        error_message = str(e)
        print("Error in get_ingredient_nutrition with following parameters: " + "id: " + str(ingredient_id) + " " + json.dumps(querystring) + " " + error_message)
        
def convert_amounts(ingredient_name, source_amount, source_unit, target_unit):
    """
    :param ingredient_name: name of ingredient
    :param source_amount: original amount
    :param source_unit: original unit
    :param target_unit: target unit
    :return: amount in target unit
    """
    querystring = {"ingredientName":ingredient_name, "sourceAmount":source_amount, "sourceUnit":source_unit, "targetUnit":target_unit}
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/convert"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"
    }
    
    try:
        response = requests.get(url=url, headers=headers, params=querystring)
        data = response.json()
        return data
    except Exception as e:
        error_message = str(e)
        print("Error in convert_amounts with following parameters: " + json.dumps(querystring) + " " + error_message)
        
def post_to_shopping_list(item):
    """
    post_to_shopping_list adds the specified item to the user's shopping list in the meal planner of the spoonacular website
    :param item: item to add to the shopping list
    """
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/mealplanner/dsky/shopping-list/items"
    
    headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
	"Content-Type": "application/json"
    }
    
    try:
        response = requests.post(url=url, headers=headers, json={"item":item})
        return f"{item} posted successfully"
    except Exception as e:
        error_message = str(e)
        print("Error in post_to_shopping_list: " + error_message)
        
def get_items_from_storage(csv_file='storage\storage_PlanAndSolve.csv'):
    """ 
    :return: items in storage in JSON format 
    """
    items = []

    try:
        # Read the CSV file
        with open(csv_file, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            
            # Iterate over each row and add it to the items list
            for row in reader:
                item = {
                    'name': row['name'] if 'name' in row else None,
                    'amount': int(row['amount']) if 'amount' in row else None,
                    'unit': row['unit'] if 'unit' in row else None,
                    'price': float(row['price']) if 'price' in row else None
                }
                items.append(item)
        
        # Convert the list of items to a dictionary
        items_dict = {'items': items}
        return json.dumps(items_dict, indent=4)
    except Exception as e:
        error_message = str(e)
        print("Error in get_items_from_storage: " + error_message)

Supporting Functions

In [6]:
def create_storage_csv(csv_file='storage\storage_SelfReflectAndPlanAndSolve.csv'):
    """ 
    create_storage_csv creates a csv file with items in storage in JSON format 
    """
    fieldnames = ['name', 'amount', 'unit', 'price']
    
    # Open the CSV file in write mode, which will create the file if it doesn't exist
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
    
    print(f"CSV file '{csv_file}' created with columns: {', '.join(fieldnames)}")
    
def clear_storage(csv_file='storage\storage_SelfReflectAndPlanAndSolve.csv'):
    """
    clear_storage clears the storage from all items
    """
    fieldnames = ['name', 'amount', 'unit', 'price']
    
    # Open the CSV file in write mode, which will truncate the file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Write the header to the file
        writer.writeheader()
    
    print(f"All items cleared from '{csv_file}', but column names are retained.")

def add_item_to_storage(name, amount, unit, price, csv_file='storage\storage_SelfReflectAndPlanAndSolve.csv'):
    """
    add_item_to_storage adds the specified item to the storage
    :param csv_file: csv_file that represents the storage
    :param name: name of item
    :param amount: amount of item
    :param unit: unit of item
    :param price: price of item
    """
    try:
        # Open the CSV file in append mode
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            
            # Append the new item as a new row in the CSV file
            writer.writerow([name, amount, unit, price])
    except Exception as e:
        error_message = str(e)
        print("Error in add_item_to_storage: " + error_message )
        
def create_log_file(log_file='data\log_file_SelfReflectAndPlanAndSolve.csv'):
    """
    create_log_file creates a log file
    """
    fieldnames = ['use_case_id', 'role', 'evaluation', 'tool_calls', 'final_answer', 'error_message', 'completion_tokens', 'prompt_tokens', 'total_tokens', 'model']
    
    # Open the CSV file in write mode, which will create the file if it doesn't exist
    with open(log_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
    
    print(f"CSV file '{log_file}' created with columns: {', '.join(fieldnames)}")
    
def log_details(use_case_id, role, evaluation, tool_calls, final_answer, error_message, completion_tokens, prompt_tokens, total_tokens, log_file='data\log_file_SelfReflectAndPlanAndSolve.csv'):
    """
    log_use_case_details logs information about the run in the log file
    """
    with open (log_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([use_case_id, role, evaluation, tool_calls, final_answer, error_message, completion_tokens, prompt_tokens, total_tokens, GPT_MODEL])


Experiment Functions

In [7]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools):
    """
    :param messages: current messages to create a ChatCompletion to
    :param tools: tools the model is equipped with
    :return: ChatCompletion response
    """
    try:
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [8]:
def execute(messages, use_case_id):
    """
    execute executes a use case initiated by the first messages being the system prompt and the user query. Tool calls are completed as long as the model decides that a tool can help in finding a solution. After that the final message is created.
    :param messages: conversation consisting of the system prompt and the user query
    :param use_case_id: use case id
    :return: complete conversation run
    """
    # Configuration for logging
    tool_calls = []
    initial_messages_length = len(messages)
    evaluation = None
    role = "Initial"
    if messages[0] == system_message_reflect:
        role = "Reflect"
    if messages[0] == system_message_solver:
        role = "Solver"
    
    # Create initial response to user request and add it to messages
    response = chat_completion_request(messages, tools)
    response_message = response.choices[0].message
    messages.append(response_message)
    
    current_tool_calls = response_message.tool_calls
    
    # Get usage data
    completion_tokens = response.usage.completion_tokens
    prompt_tokens = response.usage.prompt_tokens
    total_tokens = response.usage.total_tokens
    
    # While use case is not solved, continue creating responses
    finish_reason = response.choices[0].finish_reason
    while finish_reason == 'tool_calls':
        
        # For each tool call, call the appropriate function
        for tool in current_tool_calls:

            try:
                tool_id = tool.id
                function_name = tool.function.name
                arguments = json.loads(tool.function.arguments)
                chosen_function = eval(function_name)
                
                # Manually sleep for 0.6 sec as spoonacular only allows for 2 requests per second
                if function_name in SPOONACULAR_TOOLS:
                    time.sleep(0.6)
                
                function_result = chosen_function(**arguments)
                
                print("Chosen function: " + str({function_name}) + "(" + str(arguments) + ")" + "->" + str(function_result))                
                # For logging
                tool_calls.append({
                    "function_name": function_name,
                    "arguments": arguments,
                    "returns": function_result
                })
                
                # Append result to messages
                messages.append({
                    "role": "tool", "tool_call_id": tool_id, "name": function_name, "arguments": str(arguments), "content": str(function_result),
                })
            
            # If tool call fails, provide information and log progress
            except Exception as e:
                # Append message about failed tool call to messages
                messages.append({
                    "role": "tool", "tool_call_id": tool.id, "name": tool.function.name, "arguments": str(tool.function.arguments), "content": "Error in tool call",
                })
                error_message = str(e)
                # Try to create log entry
                try:
                    final_answer = messages[-1].content if messages else error_message
                    log_details(use_case_id, role, evaluation, tool_calls, final_answer, error_message, completion_tokens, prompt_tokens, total_tokens)
                except Exception as e:
                    print("Could not create log entry")
                    print(e)
                print("Error during a function call: " + error_message)
        
        # After gathering all results for the tool calls, generate next response
        next_response = chat_completion_request(messages, tools)
        next_response_message = next_response.choices[0].message 
        messages.append(next_response_message)
        
        # Update usage data
        completion_tokens = completion_tokens + next_response.usage.completion_tokens
        prompt_tokens = prompt_tokens + next_response.usage.prompt_tokens
        total_tokens = total_tokens + next_response.usage.total_tokens
        
        # Alter current_tool_calls with next tool calls
        current_tool_calls = next_response_message.tool_calls
        
        # Break out of while loop if next_response doesn't use further function_calls
        finish_reason = next_response.choices[0].finish_reason
    
    if role == "Reflect":
        evaluation = messages[initial_messages_length].content
    
    # Log the use case details
    log_details(use_case_id, role, evaluation, tool_calls, messages[-1].content, None, completion_tokens, prompt_tokens, total_tokens)
    
    return messages

In [9]:
def run_self_reflection_and_plan_and_solve(messages, use_case_id):
    # Make a plan
    plan = [system_message_solver, {"role": "user", "content": execute(messages, use_case_id)[-1].content}]
    # Solve the plan
    answer_to_reflect = execute(plan, use_case_id)
    # Reflect on plan
    answer_to_reflect[0] = system_message_reflect
    return execute(answer_to_reflect, use_case_id)

In [10]:
# System message planner
system_message = {"role": "system",
                  "content": "You are a planner. Your role is to generate a plan that solves the user's request. The plan should specify multiple tool calls that lead to a final answer that complies with the user's requirements. You are not supposed to execute the plan, just return the plan! The user is Martin. Martin lives in Mannheim, Germany. He is 22 years old, male, 175 centimeters tall, and weighs 70 kgs. Martin does not like to eat the same food twice a day. Martin is moderately active and his goal is to maintain his weight and muscles. He is a beginner at working out."}

In [11]:
system_message_solver = {"role": "system",
                         "content": "You are an executor. Your role is to execute a plan that is provided to you. The plan includes multiple tool calls that support you getting all the information necessary to generate a final answer. The final answer has to comply to all of the user's requirements. The user is Martin. Martin lives in Mannheim, Germany. He is 22 years old, male, 175 centimeters tall, and weighs 70 kgs. Martin does not like to eat the same food twice a day. Martin is moderately active and his goal is to maintain his weight and muscles. He is a beginner at working out."}

In [12]:
system_message_reflect = {"role": "system",
                  "content": "Your role is to evaluate the answer to a user’s query by another model. You are handed the user’s query, the tool calls the model used to answer to the user, and the final answer. Your job is to evaluate the model's performance in using tool calls effectively to generate a final answer, compliant with the user’s requirements. You can assume that the result of each function call with the specific arguments is correct, and repeating it won't change the result. If you find the performance to be sufficient, only answer with 'sufficient'. If not, make a plan on how to comply with the user’s requirements better and carry it out step by step. The user’s name is Martin, he is 22 years old, male, 175 cm tall, and weighs 70 kgs. Martin does not like to eat the same food twice a day. Martin is moderately active and his goal is to maintain his weight and muscles. He is a beginner at working out."}

In [13]:
pre_content = "Please provide a high level plan for how this user query could be solved: "

Use Case Preparation

In [14]:
CURRENT_TEMP = int(json.loads(get_temperature(8.4673098, 49.4892913))["temperature"])

In [15]:
# Set 1
def run_use_case_1():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP + 1} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend a movie starring Arnold Schwarzenegger to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 1)

def run_use_case_2():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP + 1} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, i want to watch a movie with Arnold Schwarzenegger."}]
    return run_self_reflection_and_plan_and_solve(messages, 2)

def run_use_case_3():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP + 1} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend something with Paul Mescal to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 3)

def run_use_case_4():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP - 1} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend a movie starring Arnold Schwarzenegger to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 4)

def run_use_case_5():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP - 1} degrees, I want to do a workout outside for my biceps using some equipment that is easy to bring to the park. If it isn't, recommend a movie starring Arnold Schwarzenegger to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 5)

def run_use_case_6():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP - 1} degrees, I want to do a workout outside consisting of three exercises for different muscles using some equipment that is easy to bring to the park. If it isn't, recommend a movie starring Arnold Schwarzenegger to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 6)

def run_use_case_7():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend a movie starring Arnold Schwarzenegger to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 7)

def run_use_case_8():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend a movie starring Paul Mescal to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 8)

def run_use_case_9():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} If it is warmer than {CURRENT_TEMP} degrees, I want to do a workout for my biceps using the ez-bar. If it isn't, recommend a movie starring Quentin Tarantino to watch."}]
    return run_self_reflection_and_plan_and_solve(messages, 9)

# Set 2
def run_use_case_10():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend an exercise using the ez-bar to work on my biceps. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 10)

def run_use_case_11():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend three exercises to work on my appearance. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 11)

def run_use_case_12():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend an exercise using the ez-bar to work on my biceps. Then provide some recipes that fulfill my dietary needs for today. I already had 1610kcl, 80g of protein, 220g carbs and 40g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 12)

def run_use_case_13():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on my upper body. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 13)

def run_use_case_14():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on my whole body. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 14)

def run_use_case_15():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on my upper body. Then provide some recipes that fulfill my dietary needs for today. I already had 1610kcl, 80g of protein, 220g carbs and 40g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 15)

def run_use_case_16():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on my upper body. I want to use a minimum of three different kinds of equipment. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 16)

def run_use_case_17():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on muscles in my upper body. I want to use a minimum of three different kinds of equipment. Then provide a recipe that fulfills my dietary needs for today. I already had 1931kcl, 111g of protein, 272g carbs and 55g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 17)

def run_use_case_18():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please recommend a balanced workout consisting of 6 exercises focusing on my upper body. I want to use a minimum of three different kinds of equipment. Then provide some recipes that fulfill my dietary needs for today. I already had 1610kcl, 80g of protein, 220g carbs and 40g fats."}]
    return run_self_reflection_and_plan_and_solve(messages, 18)

# Set 3
def run_use_case_19():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with recipes for the day."}]
    return run_self_reflection_and_plan_and_solve(messages, 19)

def run_use_case_20():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with enough recipes for the day."}]
    return run_self_reflection_and_plan_and_solve(messages, 20)

def run_use_case_21():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with enough recipes for the day to fulfill my protein need."}]
    return run_self_reflection_and_plan_and_solve(messages, 21)

def run_use_case_22():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with recipes for the day. Keep in mind, my goal is to maintain my weight."}]
    return run_self_reflection_and_plan_and_solve(messages, 22)

def run_use_case_23():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with recipes for the day. Make sure my macros for the day are met!"}]
    return run_self_reflection_and_plan_and_solve(messages, 23)

def run_use_case_24():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk the whole day. Please provide me with enough recipes for the day to meet my macros!"}]
    return run_self_reflection_and_plan_and_solve(messages, 24)

def run_use_case_25():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk until dinner. After that I will go to a spinning class. Please provide me with recipes for the day."}]
    return run_self_reflection_and_plan_and_solve(messages, 25)

def run_use_case_26():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk until dinner. After that I will go to my spinning class which is really exhausting for me. Please provide me with recipes for the day."}]
    return run_self_reflection_and_plan_and_solve(messages, 26)

def run_use_case_27():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tomorrow will be a very busy day at the office, so I will sit at my desk until dinner. After that I will go to a spinning class. Please provide me with recipes for the day. Because the spinning class is so exhausting I want to have a late evening snack with at least 15g of protein."}]
    return run_self_reflection_and_plan_and_solve(messages, 27)

# Set 4
def run_use_case_28():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 28)

def run_use_case_29():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes Suzette for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 29)

def run_use_case_30():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes Suzette and oatmeal with water and one cup of blueberries for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 30)

def run_use_case_31():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had one serving of Crepes, an Apple, and 1 cup of Walnuts for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 31)

def run_use_case_32():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had one serving of Crepes, an Apple, and two servings of Scrambled eggs for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 32)

def run_use_case_33():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had one serving of Crepes, an Apple, and two servings of British Muffins for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 33)

def run_use_case_34():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today me and my wife shared three servings of Crepes for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 34)

def run_use_case_35():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today me and my wife shared three servings of Crepes equally for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 35)

def run_use_case_36():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today me and my wife equally shared three servings of Crepes Suzette for breakfast, how much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 36)

def run_use_case_37():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today me and my wife had three servings of Crepes for breakfast. I ate half of them. How much protein do I still need to eat today?"}]
    return run_self_reflection_and_plan_and_solve(messages, 37)

def run_use_case_38():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes for breakfast, how much percent of my daily calories did I already eat?"}]
    return run_self_reflection_and_plan_and_solve(messages, 38)

def run_use_case_39():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes Suzette for breakfast, how much percent of my daily calories did I already eat?"}]
    return run_self_reflection_and_plan_and_solve(messages, 39)

def run_use_case_40():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Scrambled eggs for breakfast, how much percent of my daily calories did I already eat?"}]
    return run_self_reflection_and_plan_and_solve(messages, 40)

def run_use_case_41():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes for breakfast, how much percent of my daily calories did I already eat? To cover at least 20 percent of the remaining calories please provide me with a recipe of the same cuisine as Crepes"}]
    return run_self_reflection_and_plan_and_solve(messages, 41)

def run_use_case_42():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes Suzette for breakfast, how much percent of my daily calories did I already eat? To cover at least 20 percent of the remaining calories please provide me with a recipe of the same cuisine as Crepes"}]
    return run_self_reflection_and_plan_and_solve(messages, 42)

def run_use_case_43():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Today I had two servings of Crepes Suzette for breakfast, how much percent of my daily calories did I already eat? To cover between 15 and 25 percent of the remaining calories please provide me with a recipe of the same cuisine as Crepes"}]
    return run_self_reflection_and_plan_and_solve(messages, 43)

# Set 5
def run_use_case_44():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, and show the difference in macros."}]
    return run_self_reflection_and_plan_and_solve(messages, 44)

def run_use_case_45():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? Half of my friends are vegetarian. The rest wants to eat meat though. Recommend a vegetarian alternative to meat, and show the difference in macros."}]
    return run_self_reflection_and_plan_and_solve(messages, 45)

def run_use_case_46():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping and do not have a shopping list so far. For when should I invite my friends over for dinner? Half of my friends are vegetarian. The rest wants to eat meat though. Recommend a vegetarian alternative to meat, and show the difference in macros."}]
    return run_self_reflection_and_plan_and_solve(messages, 46)

def run_use_case_47():
    clear_storage()
    add_item_to_storage("onion", 6, "piece", 1.2)
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list."}]
    return run_self_reflection_and_plan_and_solve(messages, 47)

def run_use_case_48():
    clear_storage()
    add_item_to_storage("onion", 6, "piece", 1.2)
    add_item_to_storage("garlic", 3, "gloves", 0.2)
    add_item_to_storage("tofu", 600, "g", 2)
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list. I think i have tofu at home though."}]
    return run_self_reflection_and_plan_and_solve(messages, 48)

def run_use_case_49():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? Half of my friends are vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list. We want to do half of the meal in vegetarian and in another pot half of it with meat. One friend also is lactose intolerant is this a problem? If yes how should we alter the vegetarian pot to account for his intolerance"}]
    return run_self_reflection_and_plan_and_solve(messages, 49)

# 3 gloves of garlic
def run_use_case_50():
    clear_storage()
    add_item_to_storage("Onion", 5, "pieces", 4.5)
    add_item_to_storage("Garlic (with skin)", 3, "pieces", 4.5)
    add_item_to_storage("Olive Oil", 1, "l", 2)
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list. Also take into account what I already have at home."}]
    return run_self_reflection_and_plan_and_solve(messages, 50)

# onion, garlic only 3
def run_use_case_51():
    clear_storage()
    add_item_to_storage("Red Onion", 5, "pieces", 4.5)
    add_item_to_storage("Garlic (with skin)", 3, "gloves", 4.5)
    add_item_to_storage("Olive Oil", 1, "l", 2)
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list. Also take into account what I already have at home."}]
    return run_self_reflection_and_plan_and_solve(messages, 51)

# no onion, 3 gloves of garlic, no pasta
def run_use_case_52():
    clear_storage()
    add_item_to_storage("Onion", 5, "pieces", 4.5)
    add_item_to_storage("Garlic (with skin)", 3, "gloves", 4.5)
    add_item_to_storage("Olive Oil", 1, "l", 2)
    add_item_to_storage("Fusilli", 1, "kg", 4)
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Tonight I want to cook this: https://www.acozykitchen.com/spaghetti-meatballs. Before starting to cook at 7pm, I need to go grocery shopping. For when should I invite my friends over for dinner? One friend is vegetarian. Recommend a vegetarian alternative to meat, show the difference in macros, and add everything I need to buy to the shopping list. Also take into account what I already have at home."}]
    return run_self_reflection_and_plan_and_solve(messages, 52)

# Set 6
def run_use_case_53():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} For the next three days I need a meal plan with three meals a day. I like to eat meat once a day, however I don’t want to eat the same lunch or dinner twice and also I dislike eating the same kind of meat twice in three days."}]
    return run_self_reflection_and_plan_and_solve(messages, 53)

def run_use_case_54():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} For the next three days I need a meal plan with three meals a day. I like to eat meat once a day, however I don't like to eat the same kind of meat twice during the three days. During one day I dont like to eat the same lunch or dinner twice."}]
    return run_self_reflection_and_plan_and_solve(messages, 54)

def run_use_case_55():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} For the next three days I need a meal plan with three meals a day. I like to eat meat once a day, however I don’t want to eat the same lunch or dinner twice and also I dislike eating the same kind of meat twice in three days. Also I had way too much cheese the last days, so please don't include cheese in the recipes."}]
    return run_self_reflection_and_plan_and_solve(messages, 55)

def run_use_case_56():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. In the next three days I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. I don’t want to eat the same lunch or dinner twice."}]
    return run_self_reflection_and_plan_and_solve(messages, 56)

def run_use_case_57():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. In the next three days I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. I don’t want to eat the same lunch or dinner twice. For this time it is most important to me to reach my carbs goal, less important to meet the rest of the macros."}]
    return run_self_reflection_and_plan_and_solve(messages, 57)


def run_use_case_58():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. In the next three days I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. I don’t want to eat the same lunch or dinner twice. For this time it is most important to me to reach my carbs goal, less important to meet the rest of the macros. I am also willing to eat not only three times a day. Just make sure my carbs are met."}]
    return run_self_reflection_and_plan_and_solve(messages, 58)

def run_use_case_59():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. In the next three days I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. I will also do heavy endurance training in that time. I don’t want to eat the same lunch or dinner twice."}]
    return run_self_reflection_and_plan_and_solve(messages, 59)

def run_use_case_60():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. In the next three days I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. I will also do heavy endurance training during these three days. I don’t want to eat the same lunch or dinner twice. Make sure that i hit my macro goal!"}]
    return run_self_reflection_and_plan_and_solve(messages, 60)

def run_use_case_61():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I need a meal plan for the next three days. I want to load up on carbs so I want to eat 20 percent more carbs than I normally would. On the three days I will be very active in preparing for a triathlon, therefore I will do heavy endurance training every day. Also I don’t want to eat the same lunch or dinner twice"}]
    return run_self_reflection_and_plan_and_solve(messages, 61)

# Set 7
def run_use_case_62():
    clear_storage()
    add_item_to_storage("Carrot", 7, "piece", 1.2)
    add_item_to_storage("Spinach", 500, "g", 2.99)
    add_item_to_storage("Broccoli", 3, "kg", 3)
    add_item_to_storage("Potato", 5, "lb", 4.75)
    add_item_to_storage("Tomato", 20, "piece", 3.75)
    add_item_to_storage("Onion", 5, "ounce", 1.20)
    add_item_to_storage("Cucumber", 1, "piece", 0.99)
    add_item_to_storage("Bell Pepper", 3, "piece", 2.75)
    add_item_to_storage("Garlic", 20, "g", 1.1)
    add_item_to_storage("Zucchini", 1, "kg", 4.00)
    # Fruit
    add_item_to_storage("Apple", 3, "kg", 6.20)
    add_item_to_storage("Banana", 5, "piece", 3.75)
    add_item_to_storage("Orange", 1, "kg", 7.00)
    add_item_to_storage("Strawberry", 500, "g", 6.00)
    add_item_to_storage("Blueberry", 450, "g", 4.50)
    add_item_to_storage("Pineapple", 1, "piece", 5.25)
    add_item_to_storage("Grapes", 20, "g", 1.00)
    add_item_to_storage("Watermelon", 1, "kg", 4.50)
    add_item_to_storage("Peach", 2, "lb", 5.50)
    add_item_to_storage("Mango", 2, "kg", 20.25)
    # Dairy
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Cheese", 1, "kg", 12.00)
    add_item_to_storage("Yogurt", 2, "kg", 4.50)
    # Nuts
    add_item_to_storage("Almonds", 1, "kg", 20.00)
    add_item_to_storage("Cashews", 4, "lb", 12.50)
    # Meat
    add_item_to_storage("Chicken Breast", 500, "g", 8.00)
    add_item_to_storage("Beef Steak", 1, "kg", 12.50)
    add_item_to_storage("Pork Chops", 1, "kg", 15.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please provide me with a recipe using the least expensive vegetable and meat in my storage."}]
    return run_self_reflection_and_plan_and_solve(messages, 62)

def run_use_case_63():
    clear_storage()
    add_item_to_storage("Carrot", 7, "piece", 1.2)
    add_item_to_storage("Spinach", 500, "g", 2.99)
    add_item_to_storage("Broccoli", 3, "kg", 3)
    add_item_to_storage("Potato", 5, "lb", 4.75)
    add_item_to_storage("Tomato", 20, "piece", 3.75)
    add_item_to_storage("Onion", 5, "ounce", 1.20)
    add_item_to_storage("Cucumber", 1, "piece", 0.99)
    add_item_to_storage("Bell Pepper", 3, "piece", 2.75)
    add_item_to_storage("Garlic", 20, "g", 1.1)
    add_item_to_storage("Zucchini", 1, "kg", 4.00)
    # Fruit
    add_item_to_storage("Apple", 3, "kg", 6.20)
    add_item_to_storage("Banana", 5, "piece", 3.75)
    add_item_to_storage("Orange", 1, "kg", 7.00)
    add_item_to_storage("Strawberry", 500, "g", 6.00)
    add_item_to_storage("Blueberry", 450, "g", 4.50)
    add_item_to_storage("Pineapple", 1, "piece", 5.25)
    add_item_to_storage("Grapes", 20, "g", 1.00)
    add_item_to_storage("Watermelon", 1, "kg", 4.50)
    add_item_to_storage("Peach", 2, "lb", 5.50)
    add_item_to_storage("Mango", 2, "kg", 20.25)
    # Dairy
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Cheese", 1, "kg", 12.00)
    add_item_to_storage("Yogurt", 2, "kg", 4.50)
    # Nuts
    add_item_to_storage("Almonds", 1, "kg", 20.00)
    add_item_to_storage("Cashews", 4, "lb", 12.50)
    # Meat
    add_item_to_storage("Chicken Breast", 500, "g", 8.00)
    add_item_to_storage("Beef Steak", 1, "kg", 12.50)
    add_item_to_storage("Pork Chops", 1, "kg", 15.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} please help me prepare a three-course meal for my family. Each course should contain at least one ingredient from my storage. Please choose the most expensive vegetable, the most expensive meat and the most expensive fruit."}]
    return run_self_reflection_and_plan_and_solve(messages, 63)

def run_use_case_64():
    clear_storage()
    add_item_to_storage("Carrot", 7, "piece", 1.2)
    add_item_to_storage("Spinach", 500, "g", 2.99)
    add_item_to_storage("Broccoli", 3, "kg", 3)
    add_item_to_storage("Potato", 5, "lb", 4.75)
    add_item_to_storage("Tomato", 20, "piece", 3.75)
    add_item_to_storage("Onion", 5, "ounce", 1.20)
    add_item_to_storage("Cucumber", 1, "piece", 0.99)
    add_item_to_storage("Bell Pepper", 3, "piece", 2.75)
    add_item_to_storage("Garlic", 20, "g", 1.1)
    add_item_to_storage("Zucchini", 1, "kg", 4.00)
    # Fruit
    add_item_to_storage("Apple", 3, "kg", 6.20)
    add_item_to_storage("Banana", 5, "piece", 3.75)
    add_item_to_storage("Orange", 1, "kg", 7.00)
    add_item_to_storage("Strawberry", 500, "g", 6.00)
    add_item_to_storage("Blueberry", 450, "g", 4.50)
    add_item_to_storage("Pineapple", 1, "piece", 5.25)
    add_item_to_storage("Grapes", 20, "g", 1.00)
    add_item_to_storage("Watermelon", 1, "kg", 4.50)
    add_item_to_storage("Peach", 2, "lb", 5.50)
    add_item_to_storage("Mango", 2, "kg", 20.25)
    # Dairy
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Cheese", 1, "kg", 12.00)
    add_item_to_storage("Yogurt", 2, "kg", 4.50)
    # Nuts
    add_item_to_storage("Almonds", 1, "kg", 20.00)
    add_item_to_storage("Cashews", 4, "lb", 12.50)
    # Meat
    add_item_to_storage("Chicken Breast", 500, "g", 8.00)
    add_item_to_storage("Beef Steak", 1, "kg", 12.50)
    add_item_to_storage("Pork Chops", 1, "kg", 15.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} please help me prepare a three-course meal for my family. Each course should contain at least one ingredient from my storage. Please choose the most expensive vegetable, the most expensive meat and the most expensive fruit. Also prepare a shopping list for me."}]
    return run_self_reflection_and_plan_and_solve(messages, 64)

def run_use_case_65():
    clear_storage()
    add_item_to_storage("Carrot", 21, "piece", 1.2)  # 427g so around 2.4 per kg
    add_item_to_storage("Spinach", 500, "g", 2.99)  #6 per kg
    add_item_to_storage("Broccoli", 3, "kg", 3)  # 1 per kg
    add_item_to_storage("Potato", 5, "lb", 4.75)  # 2 per kg
    add_item_to_storage("Tomato", 20, "piece", 3.75)
    add_item_to_storage("Onion", 5, "ounce", 1.20)  # 8,5 per kg
    add_item_to_storage("Cucumber", 1, "piece", 0.99)
    add_item_to_storage("Bell Pepper", 3, "piece", 2.75)
    add_item_to_storage("Garlic", 20, "g", 1.1)  # ....
    add_item_to_storage("Zucchini", 1, "kg", 4.00)  # 4 per kg

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please provide me with a recipe using the least expensive vegetable relative to weight in my storage."}]
    return run_self_reflection_and_plan_and_solve(messages, 65)

def run_use_case_66():
    clear_storage()
    add_item_to_storage("Carrot", 21, "piece", 1.2)  # 427g so around 2.4 per kg
    add_item_to_storage("Spinach", 500, "g", 2.99)  #6 per kg
    add_item_to_storage("Broccoli", 3, "kg", 3)  # 1 per kg
    add_item_to_storage("Potato", 5, "lb", 4.75)  # 2 per kg
    add_item_to_storage("Tomato", 20, "piece", 3.75)
    add_item_to_storage("Onion", 5, "ounce", 1.20)  # 8,5 per kg
    add_item_to_storage("Cucumber", 1, "piece", 0.99)
    add_item_to_storage("Bell Pepper", 3, "piece", 2.75)
    add_item_to_storage("Garlic", 20, "g", 1.1)  # ....
    add_item_to_storage("Zucchini", 1, "kg", 4.00)  # 4 per kg

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} Please provide me with a recipe using the least expensive vegetable relative to weight in my storage. For items that are in storage in pieces, calculate their weight first."}]
    return run_self_reflection_and_plan_and_solve(messages, 66)

def run_use_case_67():
    clear_storage()
    add_item_to_storage("Kiwi", 1, "kg", 6.00)
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Yogurt", 1, "kg", 4.50)
    add_item_to_storage("Cashews", 3, "lb", 12.50)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} How much percent of my daily protein need can I cover with a recipe with the protein densest product in my storage?"}]
    return run_self_reflection_and_plan_and_solve(messages, 67)

def run_use_case_68():
    clear_storage()
    add_item_to_storage("Kiwi", 1, "kg", 6.00)
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Yogurt", 1, "kg", 4.50)
    add_item_to_storage("Cashews", 3, "lb", 12.50)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} How much percent of my daily protein need can I cover with a recipe with the calorie densest product in my storage?"}]
    return run_self_reflection_and_plan_and_solve(messages, 68)

def run_use_case_69():
    clear_storage()
    add_item_to_storage("Kiwi", 1, "kg", 6.00)
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Yogurt", 1, "kg", 4.50)
    add_item_to_storage("Cashews", 3, "lb", 12.50)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} During my bike ride on Saturday I need to eat 30g of carbs to fuel. However I want to carry as little weight as possible with me on the ride. Also I will not have time to go grocery shopping before."}]
    return run_self_reflection_and_plan_and_solve(messages, 69)

def run_use_case_70():
    clear_storage()
    add_item_to_storage("Flour", 5, "kg", 4.00)
    add_item_to_storage("Butter", 1, "lb", 6.00)
    add_item_to_storage("Milk", 2000, "g", 3.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I either want to eat one serving of Crepes, Waffles. Which one should I make to reduce the total weight of my storage the most?"}]
    return run_self_reflection_and_plan_and_solve(messages, 70)

def run_use_case_71():
    clear_storage()
    add_item_to_storage("Flour", 5, "kg", 4.00)
    add_item_to_storage("Butter", 1, "lb", 6.00)
    add_item_to_storage("Milk", 2000, "g", 3.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I either want to make a recipe for one serving of Crepes or Waffles. Which one should I make to reduce the total weight of my storage the most?"}]
    return run_self_reflection_and_plan_and_solve(messages, 71)

def run_use_case_72():
    clear_storage()
    add_item_to_storage("Flour", 5, "kg", 4.00)
    add_item_to_storage("Butter", 1, "lb", 6.00)
    add_item_to_storage("Milk", 2000, "g", 3.00)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} I either want to make a recipe for Crepes or Waffles. Which one should I make to reduce the total weight of my storage the most and what would the differences in macros be?"}]
    return run_self_reflection_and_plan_and_solve(messages, 72)

def run_use_case_73():
    clear_storage()
    add_item_to_storage("Grapes", 20, "g", 1.00)
    add_item_to_storage("Watermelon", 1, "kg", 4.50)
    add_item_to_storage("Peach", 2, "lb", 5.50)
    add_item_to_storage("Mango", 2, "kg", 20.25)
    # Dairy
    add_item_to_storage("Milk", 2, "liters", 3.00)
    add_item_to_storage("Cheese", 1, "kg", 12.00)
    add_item_to_storage("Yogurt", 2, "kg", 4.50)
    # Nuts
    add_item_to_storage("Almonds", 1, "kg", 20.00)
    add_item_to_storage("Cashews", 4, "lb", 12.50)

    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} How many days could I maintain my weight if i don't go shopping?"}]
    return run_self_reflection_and_plan_and_solve(messages, 73)

def run_use_case_74():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} For my bike ride on Saturday i need to go buy something to fuel on the bike. The snack should have a lot of carbs and little fat and sugar. Which of the following suite these requirements best: banana, rice cake, beer, nuts or bagels?"}]
    return run_self_reflection_and_plan_and_solve(messages, 74)

def run_use_case_75():
    messages = [system_message, {"role": "user",
                                 "content": f"{pre_content} For my bike ride on Saturday i need to go buy something to fuel on the bike. The snack should have a lot of carbs and little fat and sugar. Out of the following: banana, rice cake, beer, nuts or bagels? Which has the highest carbs to weight ratio?"}]
    return run_self_reflection_and_plan_and_solve(messages, 75)

In [16]:
# Function to run all use case functions in a loop
def run_all_use_cases():
    for i in range(1, 76):  # Loop 
        func_name = f"run_use_case_{i}"  # Construct function name
        func = globals().get(func_name)  # Get function from global scope
        if func and callable(func):  # Check if it exists and is callable
            try:
                func()  # Call the function
            except Exception as e:
                print(str(e))
                log_details(i, None, None, "Repeat as error occurred", str(e), None, None, None, None)

Run Experiment